In [1]:
import glob
import os
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.utils import shuffle

In [3]:
df = pd.read_csv("data.csv")

In [4]:
df.head()

,a1,b1,c1,d1,e1,f1,g1,h1,a2,b2,...,to_h7,to_a8,to_b8,to_c8,to_d8,to_e8,to_f8,to_g8,to_h8,good_move
0,R,N,B,Q,K,B,N,R,P,P,...,0,0,0,0,0,0,0,0,0,0
1,R,N,B,Q,K,B,N,R,P,P,...,0,0,0,0,0,0,0,0,0,0
2,R,N,B,Q,K,B,N,R,P,P,...,0,0,0,0,0,0,0,0,0,0
3,R,N,B,Q,K,B,N,R,P,P,...,0,0,0,0,0,0,0,0,0,0
4,R,N,B,Q,K,B,N,R,P,P,...,0,0,0,0,0,0,0,0,0,0


In [7]:
df_g = df.loc[df["good_move"] == 1]

In [8]:
df_g

,a1,b1,c1,d1,e1,f1,g1,h1,a2,b2,...,to_h7,to_a8,to_b8,to_c8,to_d8,to_e8,to_f8,to_g8,to_h8,good_move
19,R,N,B,Q,K,B,N,R,P,P,...,0,0,0,0,0,0,0,0,0,1
49,R,N,B,Q,K,B,N,R,P,P,...,0,0,0,0,0,0,0,0,0,1
87,R,N,B,Q,K,B,N,R,P,P,...,0,0,0,0,0,0,0,0,0,1
126,R,None,B,Q,K,B,N,R,P,P,...,0,0,0,0,0,0,0,0,0,1
170,R,None,None,Q,K,B,N,R,P,P,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1685391,None,None,None,None,None,None,K,None,P,None,...,0,0,0,0,0,0,0,0,0,1
1685425,None,None,None,None,None,None,K,None,P,None,...,0,0,0,0,0,0,0,0,0,1
1685457,None,None,None,None,None,None,K,None,P,None,...,0,0,0,0,0,0,0,0,0,1
1685490,None,None,None,None,None,None,K,None,P,None,...,0,0,0,0,0,1,0,0,0,1


In [9]:
df_b = df.loc[df["good_move"] == 0]


In [11]:
df_b = shuffle(df_b)
df_b_train = df_b[:60000]

In [15]:
datasets =[ df_b_train , df_g]

In [18]:
datasets = pd.concat(datasets)
datasets

,a1,b1,c1,d1,e1,f1,g1,h1,a2,b2,...,to_h7,to_a8,to_b8,to_c8,to_d8,to_e8,to_f8,to_g8,to_h8,good_move
1632864,None,None,None,None,None,R,K,None,P,B,...,0,0,0,0,0,0,0,0,0,0
251265,None,None,None,None,None,None,None,R,P,r,...,0,0,0,0,0,0,0,0,0,0
162444,None,None,None,None,None,None,None,r,None,None,...,0,0,0,0,0,0,0,0,0,0
440222,R,None,None,None,K,None,None,R,P,P,...,0,0,0,0,0,0,0,0,0,0
29101,R,None,B,Q,None,R,K,None,P,None,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1685391,None,None,None,None,None,None,K,None,P,None,...,0,0,0,0,0,0,0,0,0,1
1685425,None,None,None,None,None,None,K,None,P,None,...,0,0,0,0,0,0,0,0,0,1
1685457,None,None,None,None,None,None,K,None,P,None,...,0,0,0,0,0,0,0,0,0,1
1685490,None,None,None,None,None,None,K,None,P,None,...,0,0,0,0,0,1,0,0,0,1


In [19]:
datasets = shuffle(datasets)

In [20]:
from sklearn.model_selection import train_test_split
train, test, = train_test_split(datasets, test_size=0.2)
train = train[:10000]

In [21]:
test = test[:1000]

In [22]:
features = list(train.iloc[:, 0:192].columns)

In [23]:
X = train[features]
y = train['good_move']

In [24]:
categorical_columns = list(X.iloc[:, 0:63].columns)
numerical_columns = list(X.iloc[:, 64:192].columns)
feature_columns = []

for feature_name in categorical_columns:
    vocabulary = X[feature_name].unique()
    feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))


for feature_name in numerical_columns:
    feature_columns.append(tf.feature_column.numeric_column(feature_name,dtype = tf.float32))

In [25]:
def make_input_fn(data_df, label_df, num_epochs = 10, shuffle = True, batch_size = 32):
    def input_function():
        ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
        if shuffle:
            ds = ds.shuffle(1000)
        ds = ds.batch(batch_size).repeat(num_epochs)
        return ds
    return input_function

In [36]:
def split_into_batches(df, batch_size=10000):
    nb_rows = len(df.index)
    intervals = []
    
    for i in range(0, nb_rows + 1, batch_size):
        intervals.append(i)
  
    if(intervals[-1] != nb_rows):
        intervals.append(nb_rows)
  
    batches_X = []
    batches_y = []
  
    for i in range(0, len(intervals) - 1):
        batches_X.append(df.iloc[intervals[i]:intervals[i + 1], :][features])
        batches_y.append(df.iloc[intervals[i]:intervals[i + 1], :]['good_move'])

    return batches_X, batches_y

In [37]:
batches_X, batches_y = split_into_batches(train)

In [38]:
xval, yval = split_into_batches(test)

In [39]:
len(train)

10000

In [40]:
yval[0].sum()

458

In [41]:
len(batches_y[0])

10000

In [42]:
batches_y[0].sum()

4627

In [43]:
##initialize a classifier
linear_est = tf.estimator.LinearClassifier(feature_columns = feature_columns, model_dir='model_final')


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'model_final', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0xa457a7860>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [44]:
dnn_est = tf.estimator.DNNLinearCombinedClassifier(
    dnn_feature_columns=feature_columns,
    dnn_optimizer='Adam',
    dnn_hidden_units=[20],
    dnn_activation_fn=tf.nn.relu,
    n_classes=2
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/_t/5zy699z92c39l68twj0fg6mm0000gn/T/tmptzika8ix', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0xa457a7f60>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [45]:
input_functions = []
for df_X, df_y in zip(batches_X, batches_y):
    print(df_X)
    print(type(df_X))
    input_functions.append(make_input_fn(df_X, df_y))

           a1    b1    c1    d1    e1    f1    g1    h1    a2    b2  ...  \
193632   None  None     R  None  None  None     K  None     P  None  ...   
1428817     R  None     B     Q  None     R     K  None     P     P  ...   
619464      R  None     K  None  None  None  None     R  None  None  ...   
492642   None  None  None     R     R  None     K  None  None  None  ...   
259169      R     N     B     Q     K     B     N     R     P     P  ...   
...       ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   
1656382     R  None     B  None     K  None  None     R     P     P  ...   
1015091     R     N     B     Q     K     B     N     R     P     P  ...   
722016   None  None  None  None     R     K  None  None     P     P  ...   
1600800     R     N     B     Q  None     R     K  None     P  None  ...   
634746      R     N     B     Q     K     B  None     R     P     P  ...   

        to_g7 to_h7 to_a8 to_b8 to_c8 to_d8 to_e8 to_f8 to_g8 to_h8  
193632      0    

In [46]:
type(xval[0])

pandas.core.frame.DataFrame

In [47]:
type(batches_X)

list

In [48]:
# train the model on all the input functions
i = 1
for input_function in input_functions:
    print('<======================================== NEW BATCH ========================================>')
    print('Batch: ' + str(i))
    i = i + 1
    linear_est.train(input_function)
  
 # save the model
serving_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(
  tf.feature_column.make_parse_example_spec(feature_columns))



<======================================== NEW BATCH ========================================>
Batch: 1
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into model_final/model.ckpt.
INFO:tensorflow:loss = 0.6931472, step = 0
INFO:tensorflow:global_step/sec: 3.30927
INFO:tensorflow:loss = 0.66683507

In [49]:
estimator_base_path = 'model_final'
estimator_path = linear_est.export_saved_model(estimator_base_path, serving_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Signatures INCLUDED in export for Classify: ['serving_default', 'classification']
INFO:tensorflow:Signatures INCLUDED in export for Regress: ['regression']
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['predict']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Restoring parameters from model_final/model.ckpt-3130
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: model_final/temp-b'1620661902'/saved_model.pb


In [50]:
## validation

In [51]:
eval_input_fn = make_input_fn(xval[0], yval[0])

In [52]:
result = linear_est.evaluate(eval_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-05-10T17:52:30Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_final/model.ckpt-3130
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2021-05-10-17:53:28
INFO:tensorflow:Saving dict for global step 3130: accuracy = 0.589, accuracy_baseline = 0.542, auc = 0.630104, auc_precision_recall = 0.5569835, average_loss = 0.69672614, global_step = 3130, label/mean = 0.458, loss = 0.69772756, precision = 0.539629, prediction/mean = 0.5466113, recall = 0.69868994
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 3130: model_final/model.ckpt-3130
